In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""   
 CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd 
 TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH pmat_plant AS (
    SELECT DISTINCT 
        plant store_id, 
        mat_plant product_id 
    FROM {catalog_name}.udp_wcm_silver_sap_bw.sap_pmat_plant
    WHERE is_delete = 0
),
temp_0 AS (
    SELECT 
        DISTINCT 
        a.store_id,
        b.product_id,
        a.assortment,
        c.assortment_class,
        c.assortment_class_desc,
        b.vendor_id,
        a.is_delete store_delete,
        a.block_by_site,
        b.is_delete product_delete,
        b.block_by_article,
        IF(a.last_update_date > b.last_update_date,a.last_update_date,b.last_update_date) last_update_date,
        IF(p.store_id IS NULL, 0, 1) block_by_store_product,
        IF(c.status = 'ACTIVE', 0, 1) block_by_assortment,
        IF(c.name LIKE '1%' OR c.assortment IN ('VM_NVL','VMP_NVLMB','VMP_NVLMN'), 0, 1) block_by_assortment_ooa, 
        IF(a.assortment NOT LIKE 'EX_%', 0, 1) block_by_ex,
        IF(a.start_date > b.start_date, a.start_date, b.start_date) date_from,
        IF(a.end_date > b.end_date, b.end_date, a.end_date) date_to
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_store_scd a
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_article_scd b 
            ON a.assortment = b.assortment 
            AND a.end_date >= b.start_date 
            AND a.start_date <= b.end_date 
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_desc c ON a.assortment = c.assortment
        LEFT JOIN pmat_plant p ON a.store_id = p.store_id AND b.product_id = p.product_id
),
temp_1 AS (
    SELECT 
        *, IFNULL(LEAD(date_from) OVER (PARTITION BY assortment, product_id, store_id ORDER BY last_update_date, date_from),'9999-12-31') AS date_from_to 
    FROM temp_0
),
temp_2 AS (
    SELECT 
        a.assortment,	
        a.assortment_class,
        a.assortment_class_desc,
        a.product_id,	
        a.store_id,
        a.vendor_id,
        a.date_from, 
        a.last_update_date,
        IF(a.date_from_to < a.date_to, a.date_from_to, a.date_to) date_to,
        a.store_delete,
        a.block_by_site,
        a.product_delete,
        a.block_by_article,
        a.block_by_store_product,
        a.block_by_assortment,
        a.block_by_ex,
        a.block_by_assortment_ooa
    FROM temp_1 a 
),
temp_3 AS (
    SELECT *, 
        LEAD(date_from) OVER (PARTITION BY assortment, product_id, store_id ORDER BY last_update_date) date_from_to, 
        LAG(date_to,1) OVER (PARTITION BY assortment, product_id, store_id ORDER BY last_update_date) date_lag_to_to, 
        LAG(date_from,1) OVER (PARTITION BY assortment, product_id, store_id ORDER BY last_update_date) date_lag_from
    FROM temp_2
),
temp_4 AS (
    SELECT 
        a.assortment,	
        a.assortment_class,
        a.assortment_class_desc,
        a.product_id,	
        a.store_id,
        a.vendor_id,
        a.date_from , 
        a.last_update_date,
        a.date_to,
        a.store_delete,
        a.block_by_site,
        a.product_delete,
        a.block_by_article,
        a.block_by_store_product,
        a.block_by_assortment,
        a.block_by_ex,
        a.block_by_assortment_ooa,
        IF(date_from = date_lag_to_to, NULL, date_from) date_from_new,
        IF(date_to = date_from_to, NULL, date_to) date_to_new
    FROM temp_3 a 
),
temp_5 AS (
    SELECT 
        *, LEAD(date_to) OVER (PARTITION BY assortment, product_id, store_id ORDER BY last_update_date) date_to_to 
    FROM temp_4 
    WHERE date_from_new IS NOT NULL OR date_to_new IS NOT NULL
)
SELECT 
    a.assortment,	
    a.assortment_class,
    a.assortment_class_desc,
    a.store_id,	
    a.vendor_id,
    a.product_id,	
    a.date_from_new start_date,
    IFNULL(a.date_to_new,date_to_to) end_date,
    a.store_delete,
    a.block_by_site,
    a.product_delete,
    a.block_by_article,
    a.block_by_store_product,
    a.block_by_assortment,
    a.block_by_assortment_ooa,
    a.block_by_ex,
    IF(
        a.store_delete = 0 
        AND a.product_delete = 0 
        AND a.block_by_site = 0 
        AND	a.block_by_article = 0 
        AND	a.block_by_assortment = 0 
        AND a.block_by_store_product = 0,
        'ACTIVE',
        IF(
            (block_by_assortment = 0 OR block_by_ex = 1),
            'INACTIVE',
            'CLEARSTOCK'
        )
    ) status,
    IF(
        a.store_delete = 0 
        AND a.product_delete = 0 
        AND a.block_by_site = 0 
        AND	a.block_by_article = 0 
        AND	a.block_by_assortment_ooa = 0 
        AND a.block_by_store_product = 0,
        'ACTIVE', 'INACTIVE'
    ) status_ooa
FROM temp_5 a
WHERE date_from_new IS NOT NULL
""")